In [9]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/v23
bak
model_gbdt_v23.ipynb
model_lgb_v23.ipynb
model_xgb_v23.ipynb
process_v23.ipynb
test.dat.v23
train.dat.v23


In [10]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
import numpy as np
import time 
from sklearn.preprocessing import OrdinalEncoder

pd.set_option('display.max_columns',100)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
output_train_path = "train.dat.%s" % suffix

test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'
output_test_path = "test.dat.%s" % suffix

print(suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

v23
process time :  2025-08-31 21:37:52


In [11]:
# debug 模式
NROWS = 10000000000000000

#  基础特征

In [12]:
df_train  = pd.read_csv(train_path, index_col=['id'], nrows=NROWS)
df_test  = pd.read_csv(test_path, index_col=['id'], nrows=NROWS)

df_tx_train  = pd.read_csv(train_tx_path, index_col=['id'], nrows=NROWS)
df_tx_test   = pd.read_csv(test_tx_path,  index_col=['id'], nrows=NROWS)

df_train_test = pd.concat([df_train, df_test], axis=0)
df_tx_train_test = pd.concat([df_tx_train, df_tx_test], axis=0)
print(df_train.shape)
print(df_test.shape)

(53480, 18)
(20054, 17)


In [13]:
df_train_test['balance_account_avg'] = df_train_test['balance'] / df_train_test['balance_accounts']
df_train_test['loan_term_avg'] = df_train_test['loan'] / df_train_test['term']
df_train_test['balance_accounts_ratio'] = df_train_test['balance_accounts'] / df_train_test['total_accounts']

#log 处理
df_train_test['loan_log'] = df_train_test['loan'].apply(lambda x : np.log(x))
df_train_test['balance_account_avg_log'] = df_train_test['balance_account_avg'].apply(lambda x : np.log(x))
df_train_test['loan_term_avg_log'] = df_train_test['loan_term_avg'].apply(lambda x : np.log(x))
df_train_test['balance_accounts_ratio_log'] = df_train_test['balance_accounts_ratio'].apply(lambda x : np.log(x))
df_train_test['interest_rate_log'] = df_train_test['interest_rate'].apply(lambda x : np.log(x))

df_train_test['balance_log'] = df_train_test['balance'].apply(lambda x : np.log(x))
df_train_test['balance_limit_log'] = df_train_test['balance_limit'].apply(lambda x : np.log(x))
df_train_test['balance_accounts_log'] = df_train_test['balance_accounts'].apply(lambda x : np.log(x))

# zip code
df_train_test['zip_province']  = df_train_test['zip_code'].apply(lambda x : str(x)[:2])
df_train_test['zip_city']      = df_train_test['zip_code'].apply(lambda x : str(x)[:4])

#level 处理
df_train_test['level_hash'] = df_train_test['level'].apply(lambda x : hash(x) % 1000 )
encoder = OrdinalEncoder(categories=[['A0','A1','A2','A3','A4','A5','B0','B1','B2','B3','B4','B5','C0','C1','C2','C3','C4','C5','D0','D1','D2','D3','D4','D5','E0','E1','E2','E3','E4','E5']]) 
df_train_test['level_ord'] = encoder.fit_transform(df_train_test[['level']].values)

df_train_test['grade'] = df_train_test['level'].apply( lambda x : str(x)[0])  #提取ABCDE
df_train_test['grade'] = df_train_test['grade'].map({'A':1 , 'B':2, 'C':3, 'D':4, 'E':5})


# 分桶处理
bins_num = 50
tmp_labels = ['%d' % i for i in range(bins_num)]
df_train_test['interest_rate_cut'] = pd.cut(df_train_test['interest_rate'], bins=bins_num, labels = tmp_labels)

tmp_labels = ['%d' % i for i in range(bins_num)]
df_train_test['interest_rate_log_cut'] = pd.cut(df_train_test['interest_rate_log'], bins=bins_num, labels = tmp_labels)

# 等频
tmp_labels = ['%d' % i for i in range(bins_num)]
df_train_test['balance_cut'] = pd.qcut(df_train_test['balance'], q=bins_num, labels = tmp_labels, duplicates='drop')

df_train_test['loan_cut'] = pd.qcut(df_train_test['loan'], q=bins_num,  duplicates='drop')
loan_labels = np.unique(df_train_test['loan_cut'].values.codes).tolist()
df_train_test['loan_cut'] = pd.qcut(df_train_test['loan'], q=bins_num,  labels = loan_labels,  duplicates='drop')

tmp_labels = ['%d' % i for i in range(bins_num)]
df_train_test['balance_limit_cut'] = pd.qcut(df_train_test['balance_limit'], q=bins_num, labels = tmp_labels, duplicates='drop')

df_train_test['loan_term_avg_cut'] = pd.qcut(df_train_test['loan_term_avg'], q=bins_num,  duplicates='drop')
loan_labels = np.unique(df_train_test['loan_term_avg_cut'].values.codes).tolist()
df_train_test['loan_term_avg_cut'] = pd.qcut(df_train_test['loan_term_avg'], q=bins_num,  labels = loan_labels,  duplicates='drop')

tmp_labels = ['%d' % i for i in range(bins_num)]
df_train_test['balance_account_avg_cut'] = pd.qcut(df_train_test['balance_account_avg'], q=bins_num, labels = tmp_labels, duplicates='drop')

# 时间处理
df_train_test['record_time_format'] = df_train_test['record_time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
df_train_test['record_time_year'] = df_train_test['record_time_format'].map(lambda x : x.year)
df_train_test['record_time_month'] = df_train_test['record_time_format'].map(lambda x : x.month)      # 探索周期性
df_train_test['record_time_week'] = df_train_test['record_time_format'].map(lambda x : x.week)      # 探索周期性
df_train_test['record_time_year_month'] = df_train_test['record_time_format'].map(lambda x : x.strftime('%Y%m'))


df_train_test.drop(['record_time_format'], axis=1, inplace=True)
df_train_test

/d/GH/GitWorkSpace/bank_model_competiton/data/v23


,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,label,balance_account_avg,loan_term_avg,balance_accounts_ratio,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,zip_province,zip_city,level_hash,level_ord,grade,interest_rate_cut,interest_rate_log_cut,balance_cut,loan_cut,balance_limit_cut,loan_term_avg_cut,balance_account_avg_cut,record_time_year,record_time_month,record_time_week,record_time_year_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9,0.0,221373,1,7200,36,10.95,1238631967,0,1,1238630622,472006661,17.0,9.0,36200.0,13856.00,A4,0.0,1539.555556,200.000000,0.529412,8.881836,7.339249,5.298317,-0.635989,2.393339,9.536474,10.496814,2.197225,22,2213,785,4.0,1,10,20,40,4,43,6,42,2009,4,14,200904
1,8,10.0,311681,0,21300,36,12.95,1128212052,0,0,1161907665,763779041,17.0,9.0,20400.0,13773.00,B0,1.0,1530.333333,591.666667,0.529412,9.966462,7.333241,6.382943,-0.635989,2.561096,9.530465,9.923290,2.197225,31,3116,677,6.0,2,14,25,40,19,35,20,42,2006,10,43,200610
2,8,7.0,271562,1,10400,60,21.05,1249171509,0,0,1383958593,727143443,17.0,9.0,10800.0,2023.00,B4,0.0,224.777778,173.333333,0.529412,9.249561,5.415112,5.155217,-0.635989,3.046901,7.612337,9.287301,2.197225,27,2715,560,10.0,2,30,38,19,10,29,4,1,2013,11,45,201311
3,7,2.0,522083,0,33050,36,16.40,1172882234,0,1,1214353935,687660346,17.0,9.0,24700.0,21992.00,B3,0.0,2443.555556,918.055556,0.529412,10.405777,7.801209,6.822258,-0.635989,2.797281,9.998434,10.114559,2.197225,52,5220,24,9.0,2,21,31,45,22,38,27,46,2008,6,26,200806
4,8,3.0,101026,1,5200,36,14.35,1172882384,0,0,1240274527,322012875,17.0,9.0,5100.0,1669.00,B2,1.0,185.444444,144.444444,0.529412,8.556414,5.222755,4.972895,-0.635989,2.663750,7.419980,8.536996,2.197225,10,1010,710,8.0,2,17,28,16,1,24,1,1,2009,4,17,200904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,0,1125964800,1018224000,6.0,3.0,3818.0,2224.69,A4,NaN,741.563333,833.333333,0.500000,9.210340,6.608761,6.725434,-0.693147,2.936513,7.707373,8.247482,1.098612,60,6011,785,4.0,1,26,35,20,9,21,25,20,2005,9,36,200509
73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,0,1157068800,1054425600,6.0,6.0,5502.0,4126.71,B4,NaN,687.785000,833.333333,1.000000,9.210340,6.533476,6.725434,0.000000,3.377588,8.325236,8.612867,1.791759,60,6011,560,10.0,2,46,48,28,9,24,25,16,2006,9,35,200609
73531,0,4.0,601408,1,11000,12,24.75,1144108800,0,0,1111622400,1037404800,8.0,3.0,4844.0,2710.96,A3,NaN,903.653333,916.666667,0.375000,9.305651,6.806446,6.820744,-0.980829,3.208825,7.905058,8.485496,1.098612,60,6014,143,3.0,1,37,43,23,11,23,26,32,2005,3,12,200503


#  交易特征处理

In [14]:
df_tx_train_test['time_format'] = df_tx_train_test['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
print(df_tx_train_test)

#days_diff, tm_count, total_amount, amount_1, amount_0, total_amount_day_avg, amount_1_day_avg, amount_0_day_avg
print(df_tx_train_test.shape)

df_tx_stat = pd.DataFrame()
df_tx_stat['tx_time_max'] = df_tx_train_test.groupby('id')['time_format'].agg('max')
df_tx_stat['tx_time_min'] = df_tx_train_test.groupby('id')['time_format'].agg('min')

df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat['tx_count'] = df_tx_train_test.groupby('id')['amount'].agg('count')
df_tx_stat['total_amount'] = df_tx_train_test.groupby('id')['amount'].agg('sum')

df_tx_stat['1_amount'] = df_tx_train_test.groupby('id').apply( lambda x : x [ x['direction'] == 1]['amount'].sum())
df_tx_stat['0_amount'] = df_tx_train_test.groupby('id').apply( lambda x : x [ x['direction'] == 0]['amount'].sum())

df_tx_stat['total_amount_avg']  = df_tx_stat['total_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['1_amount_avg']  = df_tx_stat['1_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['0_amount_avg']  = df_tx_stat['0_amount'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg2']  = df_tx_stat['total_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['1_amount_avg2']  = df_tx_stat['1_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['0_amount_avg2']  = df_tx_stat['0_amount'] / df_tx_stat['tx_count'] 

#交易活跃度： 效果次数/天数
df_tx_stat['tx_count_avg']  = df_tx_stat['tx_count'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['tx_tmstp_max'] = df_tx_train_test.groupby('id')['time'].agg('max')
df_tx_stat['tx_tmstp_min'] = df_tx_train_test.groupby('id')['time'].agg('min')
df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)
df_tx_stat

             time  direction       amount         time_format
id                                                           
0      1224115200          0  8771.350000 2008-10-16 08:00:00
0      1224288000          1   310.650000 2008-10-18 08:00:00
0      1224460800          1   152.620000 2008-10-20 08:00:00
0      1225152000          1    20.490000 2008-10-28 08:00:00
0      1226793600          1   173.170000 2008-11-16 08:00:00
...           ...        ...          ...                 ...
71870  1160956800          1   493.403945 2006-10-16 08:00:00
71870  1161043200          0     9.462382 2006-10-17 08:00:00
71870  1161388800          1   222.936072 2006-10-21 08:00:00
71870  1161475200          0   222.936072 2006-10-22 08:00:00
71870  1161475200          1    20.908367 2006-10-22 08:00:00

[2364084 rows x 4 columns]
(2364084, 4)


,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min
id,,,,,,,,,,,,,,
0,163,48,71787.000000,12079.500000,59707.500000,440.411043,74.107362,366.303681,1495.562500,251.656250,1243.906250,0.294479,1238198400,1224115200
2,180,48,22406.100000,15883.720000,6522.380000,124.478333,88.242889,36.235444,466.793750,330.910833,135.882917,0.266667,1383955200,1368403200
4,169,93,51163.000000,30823.100000,20339.900000,302.739645,182.385207,120.354438,550.139785,331.431183,218.708602,0.550296,1238284800,1223683200
6,179,61,41733.770000,15385.270000,26348.500000,233.149553,85.951229,147.198324,684.160164,252.217541,431.942623,0.340782,1220227200,1204761600
7,175,66,59958.010000,22642.760000,37315.250000,342.617200,129.387200,213.230000,908.454697,343.072121,565.382576,0.377143,1201996800,1186876800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73515,94,25,9477.597620,5109.004148,4368.593472,100.825507,54.351108,46.474399,379.103905,204.360166,174.743739,0.265957,1161475200,1153353600
73517,180,272,212302.562407,126112.342194,86190.220212,1179.458680,700.624123,478.834557,780.524126,463.648317,316.875810,1.511111,1161475200,1145923200
73522,180,116,45701.276358,24957.995111,20743.281247,253.895980,138.655528,115.240451,393.976520,215.155130,178.821390,0.644444,1161475200,1145923200


# 合并特征、处理缺失值、保存结果

In [15]:
# 合并交易特征
df_concat = pd.concat([df_train_test, df_tx_stat], axis=1)
print(df_concat.shape)
df_concat.reset_index(inplace=True)
df_concat = df_concat.rename(columns={'index':'id'})
print(df_concat.columns)

# 缺失值处理
df_concat = df_concat.replace([np.inf, -np.inf], np.nan)

col_str = 'career,balance_limit_cut'
for col_name in col_str.split(','):
    value = df_concat[col_name].mode()[0]
    print(col_name, value)
    df_concat[col_name].fillna(value, inplace=True)
    
col_str = 'balance_limit,balance_log,balance_limit_log,balance_account_avg_log,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_avg,tx_tmstp_max,tx_tmstp_min' 
for col_name in col_str.split(',') : 
#     value = df_concat[col_name].mean(skipna=True)
    # 替换为0值
    value = 0
    df_concat[col_name].fillna(value,inplace=True)
print('isna \n',df_concat.isna().sum())

#TODO 离群点处理
df_result_train = df_concat.iloc[:df_train.shape[0],:]
df_result_test = df_concat.iloc[df_train.shape[0]:,:]
df_result_test = df_result_test.drop(['label'], axis=1)

#保存结果
df_result_train.to_csv(output_train_path, index=False)
df_result_test.to_csv(output_test_path, index=False)
print('train result', df_result_train.shape)
print('test result', df_result_test.shape)
print('output_path : ', output_train_path, output_test_path)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
# !ls

(73534, 59)
Index(['id', 'title', 'career', 'zip_code', 'residence', 'loan', 'term',
       'interest_rate', 'issue_time', 'syndicated', 'installment',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'level', 'label', 'balance_account_avg',
       'loan_term_avg', 'balance_accounts_ratio', 'loan_log',
       'balance_account_avg_log', 'loan_term_avg_log',
       'balance_accounts_ratio_log', 'interest_rate_log', 'balance_log',
       'balance_limit_log', 'balance_accounts_log', 'zip_province', 'zip_city',
       'level_hash', 'level_ord', 'grade', 'interest_rate_cut',
       'interest_rate_log_cut', 'balance_cut', 'loan_cut', 'balance_limit_cut',
       'loan_term_avg_cut', 'balance_account_avg_cut', 'record_time_year',
       'record_time_month', 'record_time_week', 'record_time_year_month',
       'tx_max_min_days', 'tx_count', 'total_amount', '1_amount', '0_amount',
       'total_amount_avg', '1_amount_avg', '0_amount_av

In [16]:
print('done  time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
!wc -l $output_train_path
!wc -l $output_test_path
df_train

done  time :  2025-08-31 21:38:27
53481 train.dat.v23
20055 test.dat.v23


,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,label
id,,,,,,,,,,,,,,,,,,
0,9,0.0,221373,1,7200,36,10.95,1238631967,0,1,1238630622,472006661,17.0,9.0,36200.0,13856.00,A4,0
1,8,10.0,311681,0,21300,36,12.95,1128212052,0,0,1161907665,763779041,17.0,9.0,20400.0,13773.00,B0,1
2,8,7.0,271562,1,10400,60,21.05,1249171509,0,0,1383958593,727143443,17.0,9.0,10800.0,2023.00,B4,0
3,7,2.0,522083,0,33050,36,16.40,1172882234,0,1,1214353935,687660346,17.0,9.0,24700.0,21992.00,B3,0
4,8,3.0,101026,1,5200,36,14.35,1172882384,0,0,1240274527,322012875,17.0,9.0,5100.0,1669.00,B2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53475,2,2.0,603000,1,9000,12,23.55,1172880000,0,0,1157587200,1061769600,12.0,5.0,3535.0,2595.73,A4,0
53476,0,10.0,601702,1,8000,12,30.70,1160092800,0,0,1138665600,1038268800,5.0,2.0,1965.0,1433.34,B2,0
53477,2,10.0,602808,1,10000,12,9.40,1180310400,0,0,1108771200,1087603200,12.0,5.0,7253.0,3813.79,B2,0
